<a href="https://colab.research.google.com/github/sxqqslf/TensorflowDaily/blob/master/ResearchAndExpermentation/CustomLayers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf

tfe = tf.contrib.eager
tf.enable_eager_execution()

In [4]:
layer = tf.keras.layers.Dense(100)
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))
layer(tf.zeros([10, 5]))
layer.variables
layer.kernel, layer.bias

(<tf.Variable 'dense_5/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-0.15205216,  0.09112263,  0.52300006,  0.57130677,  0.55442315,
          0.3381219 ,  0.04872102,  0.5728175 ,  0.6290496 ,  0.49454767],
        [-0.62898046,  0.00632226,  0.12202716,  0.56466943,  0.60190684,
         -0.30704758,  0.5227408 , -0.3215292 , -0.43332398, -0.28741935],
        [ 0.1736862 ,  0.0834446 ,  0.26997757,  0.6036008 ,  0.60789376,
          0.01749516,  0.3018785 ,  0.2737841 ,  0.24975419, -0.01216388],
        [-0.41952133, -0.5256582 ,  0.05941784,  0.4067685 , -0.06498981,
         -0.51020265, -0.3550736 ,  0.09461278, -0.22372615,  0.34045446],
        [-0.6129025 , -0.61579514,  0.19223958,  0.3420335 ,  0.03792858,
          0.5325301 ,  0.1551227 ,  0.26894665, -0.2094107 ,  0.3458073 ]],
       dtype=float32)>,
 <tf.Variable 'dense_5/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)

In [5]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs
    
  def build(self, input_shape):
    self.kernel = self.add_variable("kernel", 
                                    shape=[input_shape[-1].value,
                                          self.num_outputs])
    
  def call(self, input):
    return tf.matmul(input, self.kernel)
  
layer = MyDenseLayer(10)
print(layer(tf.zeros([10, 5])))
print(layer.variables)

tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(10, 10), dtype=float32)
[<tf.Variable 'my_dense_layer/kernel:0' shape=(5, 10) dtype=float32, numpy=
array([[ 0.58877844, -0.41728726,  0.18344814, -0.15834278,  0.47385627,
         0.38863915, -0.25918347,  0.15062356,  0.01844513, -0.06694192],
       [-0.13998029, -0.00076193, -0.28684634, -0.5995408 ,  0.4302761 ,
         0.25580084, -0.2599543 , -0.17705995,  0.17070252, -0.28980288],
       [ 0.40549392, -0.47508985, -0.3729284 ,  0.6218234 ,  0.40797764,
        -0.40276188,  0.3130535 , -0.23881277, -0.5844728 , -0.60917443],
       [-0.47313187,  0.558742  , -0.12160391, -0.39746588,  0.01602918,
        -0.5320636 ,  0.56211084, -0

In [9]:
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters
    
    self.conv2a = tf.keras.layers.Conv2D(filters1, (1,1))
    self.bn2a = tf.keras.layers.BatchNormalization()
    
    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()
    
    self.conv2c = tf.keras.layers.Conv2D(filters3, (1,1))
    self.bn2c = tf.keras.layers.BatchNormalization()
    
  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)
    
    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)
    
    x = self.conv2c(x)
    x = self.bn2c(x, training=training)
    
    x += input_tensor
    return tf.nn.relu(x)
 

block = ResnetIdentityBlock(1, [1, 2, 3])
print(block(tf.zeros([1, 2, 3, 3])))
print([x.name for x in block.variables])

tf.Tensor(
[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]], shape=(1, 2, 3, 3), dtype=float32)
['resnet_identity_block_3/conv2d_3/kernel:0', 'resnet_identity_block_3/conv2d_3/bias:0', 'resnet_identity_block_3/batch_normalization_3/gamma:0', 'resnet_identity_block_3/batch_normalization_3/beta:0', 'resnet_identity_block_3/conv2d_4/kernel:0', 'resnet_identity_block_3/conv2d_4/bias:0', 'resnet_identity_block_3/batch_normalization_4/gamma:0', 'resnet_identity_block_3/batch_normalization_4/beta:0', 'resnet_identity_block_3/conv2d_5/kernel:0', 'resnet_identity_block_3/conv2d_5/bias:0', 'resnet_identity_block_3/batch_normalization_5/gamma:0', 'resnet_identity_block_3/batch_normalization_5/beta:0', 'resnet_identity_block_3/batch_normalization_3/moving_mean:0', 'resnet_identity_block_3/batch_normalization_3/moving_variance:0', 'resnet_identity_block_3/batch_normalization_4/moving_mean:0', 'resnet_identity_block_3/batch_normalization_4/moving_variance:0', 

In [10]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1)),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.Conv2D(2, 1, 
                                                     padding='same'),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.Conv2D(3, (1, 1)),
                              tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1, 2, 3, 3]))

<tf.Tensor: id=691, shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>